Notebook 4: filtering to common independent SNPS, relatedness, PCA, joint calling with new dataset, applying RF on new dataset. Analyses that need to be run:
1. Work with Lindo: - *PENDING*
    - Joint calling with GGV, sample QC 
    - Use gnomAD RF (subset to variants in RF model) - doesn’t need VQSR 
    - Intersect HGDP+1kG+GGV, build a RF with 1kG+ HGDP, apply it to a new dataset (GGV) - doesn’t need VQSR 
2. PCA plots - Ally - *PENDING*
    - already implemented in R, just need to plot it in Hail
    
----------------------------------------
Further edits needed in this nb: 
- Add all paths below in part 1
- Add desc above each code block if needed 
- Clean up comments inside the code block 
- Add Hail links below each code block
- Add Ally's code for plots 

## Index
- [General Overview](#1.-General-Overview)
- [Variant Filtering and LD Pruning](#2.-Variant-Filtering-and-LD-Pruning)
- [Run PC Relate](#3.-Run-PC-Relate)
- [PCA](#4.-PCA)
    - [Function to Run PCA on Unrelated Individuals](#4a.-Function-to-Run-PCA-on-Unrelated-Individuals)
    - [Function to Project Related Individuals](#4b.-Function-to-Project-Related-Individuals)
    - [Global PCA](#4c.-Global-PCA)
    - [Subcontinental PCA](#4d.-Subcontinental-PCA)
- [Outlier Removal](#5.-Outlier-Removal)
- [Rerun PCA](#6.-Rerun-PCA)
    - [Global PCA](#6a.-Global-PCA)
    - [Subcontinental PCA](#6b.-Subcontinental-PCA)
- [Write Out Matrix Table](#7.-Write-Out-Matrix-Table)

# 1. General Overview 
The purpose of this notebook is to further filter the matrix table obtained from notebook 3, run relatedness and Principal Component Analysis (PCA), joint call with new data set, and apply RF. It contains steps on how to:

- Read in the a matrix table and run Hail common variant statistics  
- Filter using allele frequency and call rate
- Run LD pruning 
- Run relatedness and separate related and unrelated individuals
- Calculate PC scores and project samples on to a PC space  
- Run global and Subcontinental PCA and plot them 
- Remove PCA outliers (filter using sample IDs)
- Joint call with a new data set
- Build/apply RF
- Write out a matrix table 

Author: Mary T. Yohannes

1a. Import needed libraries and packages 

In [ ]:
# import hail
import hail as hl

1b. Input and output path variables to be edited by users as needed 

In [ ]:
# input 
input_path = 'gs://hgdp-1kg/hgdp_tgp/intermediate_files/pre_running_varqc.mt'

# temporary file
intermediate_path = 'gs://hgdp-1kg/hgdp_tgp/intermediate_files/filtered_n_pruned_output_updated.mt'

# pre-outlier paths for unrelated and related samples 
unrel_path = 'gs://hgdp-1kg/hgdp_tgp/unrel_updated.mt'
rel_path = 'gs://hgdp-1kg/hgdp_tgp/rel_updated.mt' 

# pre-outlier file path is missing - global & subcont pca results [here]

# outliers file 
outliers_path = 'gs://hgdp-1kg/hgdp_tgp/pca_outliers_v2.txt'

# post-outlier file path is missing - global & subcont pca results[here]

# final output paths for unrelated and related samples (post-outlier)
unrel_output = 'gs://hgdp-1kg/hgdp_tgp/datasets_for_others/lindo/ds_without_outliers/unrelated.mt'
rel_output = 'gs://hgdp-1kg/hgdp_tgp/datasets_for_others/lindo/ds_without_outliers/related.mt'

# 2. Variant Filtering and LD Pruning

<details>
<summary> - Why are we doing this? [click here] </summary>
    
At this point, we have 155,648,020 SNPs and since we need fewer number of variants (~100-300k) for PCA, we filter on:
- AF - allele frequency 
- call rate - fraction of calls neither missing nor filtered

and then run LD pruning.     
</details>

In [ ]:
# read-in the right intermediate file 
mt_filt = hl.read_matrix_table(input_path)

2a. Variant Filtering 

In [ ]:
# run Hail's common variant statistics (QC metrics) 
mt_var = hl.variant_qc(mt_filt) 

# filter to variants with AF between 0.05 & 0.95, and call rate greater than 0.999    
mt_var_filt = mt_var.filter_rows((mt_var.variant_qc.AF[0] > 0.05) & (mt_var.variant_qc.AF[0] < 0.95) & (mt_var.variant_qc.call_rate > 0.999))
print('Num of variants after filtering = ' + str(mt_var_filt.count()[0])) # 6787034 snps; this line take ~20min to run 

2b. LD Pruning

In [ ]:
# remove correlated variants 
pruned = hl.ld_prune(mt_var_filt.GT, r2=0.1, bp_window_size=500000) # ~113 min to run  
mt_var_pru_filt = mt_var_filt.filter_rows(hl.is_defined(pruned[mt_var_filt.row_key])) 
print('Num of variants after LD pruning = ' + str(mt_var_pru_filt.count()[0])) # 248634 snps

- Since the number of variants is now in the ~100-300k range, we proceed to the PCA analysis without any more adjustments.  

2c. Write out an intermediate file

In [ ]:
# the pruning step took a bit of time to run so we have to write out the filtered and pruned mt as an intermediate file
mt_var_pru_filt.write(intermediate_path, overwrite=False) # ~23 min to run

# read the intermediate file back in for subsequent analyses
mt_var_pru_filt = hl.read_matrix_table(intermediate_path) 

<details>
<summary> - More information on Hail methods and expressions [click here] </summary>

- <a href="more info https://hail.is/docs/0.2/methods/genetics.html#hail.methods.variant_qc"> More on  <i> variant_qc() </i></a>

- <a href="more info https://hail.is/docs/0.2/methods/genetics.html#hail.methods.ld_prune"> More on  <i> ld_prune() </i></a>

# 3. Run PC Relate   

In [ ]:
# compute relatedness estimates between individuals using a variant of the PC-Relate method (https://hail.is/docs/0.2/methods/relatedness.html#hail.methods.pc_relate)
# only compute the kinship statistic using:
# a minimum minor allele frequency filter of 0.05, 
# excluding sample-pairs with kinship less than 0.05, and 
# 20 principal components to control for population structure 
# a hail table is produced (~4min to run) 
relatedness_ht = hl.pc_relate(mt_var_pru_filt.GT, min_individual_maf=0.05, min_kinship=0.05, statistics='kin', k=20).key_by()

In [ ]:
# identify related individuals in pairs to remove - returns a list of sample IDs (~2hr & 22 min to run) - previous one took ~13min
related_samples_to_remove = hl.maximal_independent_set(relatedness_ht.i, relatedness_ht.j, False)

In [ ]:
# using sample IDs (col_key of the matrixTable), pick out the samples that are not found in 'related_samples_to_remove' (had 'False' values for the comparison)  
# subset the mt to those only 
mt_unrel = mt_var_pru_filt.filter_cols(hl.is_defined(related_samples_to_remove[mt_var_pru_filt.col_key]), keep=False) 

# do the same as above but this time for the samples with 'True' values (found in 'related_samples_to_remove')  
mt_rel = mt_var_pru_filt.filter_cols(hl.is_defined(related_samples_to_remove[mt_var_pru_filt.col_key]), keep=True) 

In [ ]:
# write out mts of unrelated and related samples on to the cloud 

# unrelated mt
mt_unrel.write(unrel_path, overwrite=False) 

# related mt 
mt_rel.write(rel_path, overwrite=False) 

In [ ]:
# read saved mts back in 

# unrelated mt
mt_unrel = hl.read_matrix_table(unrel_path) 

# related mt 
mt_rel = hl.read_matrix_table(rel_path) 

# 4. PCA

### 4a. Function to Run PCA on Unrelated Individuals

In [ ]:
def run_pca(mt: hl.MatrixTable, reg_name:str, out_prefix: str, overwrite: bool = False):
    """
    Runs PCA on a dataset
    :param mt: dataset to run PCA on
    :param reg_name: region name for saving output purposes
    :param out_prefix: path for where to save the outputs
    :return:
    """

    pca_evals, pca_scores, pca_loadings = hl.hwe_normalized_pca(mt.GT, k=20, compute_loadings=True)
    pca_mt = mt.annotate_rows(pca_af=hl.agg.mean(mt.GT.n_alt_alleles()) / 2)
    pca_loadings = pca_loadings.annotate(pca_af=pca_mt.rows()[pca_loadings.key].pca_af)
    pca_scores = pca_scores.transmute(**{f'PC{i}': pca_scores.scores[i - 1] for i in range(1, 21)})
    
    pca_scores.export(out_prefix + reg_name + '_scores.txt.bgz')  # save individual-level genetic region PCs
    pca_loadings.write(out_prefix + reg_name + '_loadings.ht', overwrite)  # save PCA loadings

### 4b. Function to Project Related Individuals

In [ ]:
#if running on GCS, need to add "--packages gnomad" when starting a cluster in order for the import to work  
from gnomad.sample_qc.ancestry import *

def project_individuals(pca_loadings, project_mt, reg_name:str, out_prefix: str, overwrite: bool = False):
    """
    Project samples into predefined PCA space
    :param pca_loadings: existing PCA space - unrelated samples 
    :param project_mt: matrixTable of data to project - related samples 
    :param reg_name: region name for saving output purposes
    :param project_prefix: path for where to save PCA projection outputs
    :return:
    """
    ht_projections = pc_project(project_mt, pca_loadings)  
    ht_projections = ht_projections.transmute(**{f'PC{i}': ht_projections.scores[i - 1] for i in range(1, 21)}) 
    ht_projections.export(out_prefix + reg_name + '_projected_scores.txt.bgz') # save output 
    #return ht_projections # return to user  

### 4c. Global PCA

In [ ]:
# run 'run_pca' function for global pca   
run_pca(mt_unrel, 'global', 'gs://hgdp-1kg/hgdp_tgp/pca_preoutlier/', False)

In [ ]:
# run 'project_relateds' function for global pca 
loadings = hl.read_table('gs://hgdp-1kg/hgdp_tgp/pca_preoutlier/global_loadings.ht') # read in the PCA loadings that were obtained from 'run_pca' function 
project_individuals(loadings, mt_rel, 'global', 'gs://hgdp-1kg/hgdp_tgp/pca_preoutlier/', False) 

### 4d. Subcontinental PCA 

In [ ]:
# obtain a list of the genetic regions in the dataset - used the unrelated dataset since it had more samples 
regions = mt_unrel['hgdp_tgp_meta']['Genetic']['region'].collect()
regions = list(dict.fromkeys(regions)) # 7 regions - ['EUR', 'AFR', 'AMR', 'EAS', 'CSA', 'OCE', 'MID']

In [ ]:
# set argument values 
subcont_pca_prefix = 'gs://hgdp-1kg/hgdp_tgp/pca_preoutlier/subcont_pca/subcont_pca_' # path for outputs 
overwrite = False 

In [ ]:
# run 'run_pca' function for each region - nb freezes after printing the log for AMR  
# don't restart it - just let it run and you can follow the progress through the SparkUI
# even after all the outputs are produced and the run is complete, the code chunk will seem as if it's still running (* in the left square bracket)
# can check if the run is complete by either checking the output files in the Google cloud bucket or using the SparkUI 
# after checking the desired outputs are generated and the run is done, exit the current nb, open a new session, and proceed to the next step
# ~27min to run 
for i in regions:
    subcont_unrel = mt_unrel.filter_cols(mt_unrel['hgdp_tgp_meta']['Genetic']['region'] == i)  # filter the unrelateds per region
    run_pca(subcont_unrel, i, subcont_pca_prefix, overwrite)

In [ ]:
# run 'project_relateds' function for each region (~2min to run)
for i in regions:
    loadings = hl.read_table(subcont_pca_prefix + i + '_loadings.ht') # for each region, read in the PCA loadings that were obtained from 'run_pca' function 
    subcont_rel = mt_rel.filter_cols(mt_rel['hgdp_tgp_meta']['Genetic']['region'] == i)  # filter the relateds per region 
    project_individuals(loadings, subcont_rel, i, subcont_pca_prefix, overwrite) 

# 5. Outlier Removal
#### After plotting the PCs, 22 outliers that need to be removed were identified (the table below will be completed for the final report)


| s | Genetic region | Population | Note |
| --- | --- | --- | -- |
| NA20314 | AFR | ASW | Clusters with AMR in global PCA | 
| NA20299 | - | - | - |
| NA20274 | - | - | - |
| HG01880 | - | - | - |
| HG01881 | - | - | - |
| HG01628 | - | - | - |
| HG01629 | - | - | - |
| HG01630 | - | - | - |
| HG01694 | - | - | - |
| HG01696 | - | - | - |
| HGDP00013 | - | - | - |
| HGDP00150 | - | - | - |
| HGDP00029 | - | - | - |
| HGDP01298 | - | - | - |
| HGDP00130 | CSA | Makrani | Closer to AFR than most CSA |
| HGDP01303 | - | - | - |
| HGDP01300 | - | - | - |
| HGDP00621 | MID | Bedouin | Closer to AFR than most MID |
| HGDP01270 | MID | Mozabite | Closer to AFR than most MID |
| HGDP01271 | MID | Mozabite | Closer to AFR than most MID |
| HGDP00057 | - | - | - | 
| LP6005443-DNA_B02 | - | - | - |


















	


In [ ]:
# read back in the unrelated and related mts to remove outliers and run pca 
# bucket was moved to another project so different paths are used from where these mts were previously saved 
mt_unrel_unfiltered = hl.read_matrix_table(unrel_path) # unrelated mt
mt_rel_unfiltered = hl.read_matrix_table(rel_path) # related mt

In [ ]:
# read the outliers file into a list
with hl.utils.hadoop_open(outliers_path) as file: 
    outliers = [line.rstrip('\n') for line in file]
    
# capture and broadcast the list as an expression
outliers_list = hl.literal(outliers)

In [ ]:
# remove 22 outliers 
mt_unrel = mt_unrel_unfiltered.filter_cols(~outliers_list.contains(mt_unrel_unfiltered['s']))
mt_rel = mt_rel_unfiltered.filter_cols(~outliers_list.contains(mt_rel_unfiltered['s']))

In [ ]:
# sanity check 
print('Unrelated: Before outlier removal ' + str(mt_unrel_unfiltered.count()[1]) + ' | After outlier removal ' + str(mt_unrel.count()[1]))
print('Related: Before outlier removal: ' + str(mt_rel_unfiltered.count()[1]) + ' | After outlier removal ' + str(mt_rel.count()[1]))

num_outliers = (mt_unrel_unfiltered.count()[1] - mt_unrel.count()[1]) + (mt_rel_unfiltered.count()[1] - mt_rel.count()[1])
print('Total samples removed = ' + str(num_outliers))

# 6. Rerun PCA
### - The following steps are similar to the ones prior to removing the outliers except now we are using the updated unrelated & related dataset and a new GCS bucket path to save the outputs 

### 6a. Global PCA

In [ ]:
# run 'run_pca' function for global pca - make sure the code block for the function (located above) is run prior to running this    
run_pca(mt_unrel, 'global', 'gs://hgdp-1kg/hgdp_tgp/pca_postoutlier/', False)

In [ ]:
# run 'project_relateds' function for global pca - make sure the code block for the function (located above) is run prior to running this    
loadings = hl.read_table('gs://hgdp-1kg/hgdp_tgp/pca_postoutlier/global_loadings.ht') # read in the PCA loadings that were obtained from 'run_pca' function 
project_individuals(loadings, mt_rel, 'global', 'gs://hgdp-1kg/hgdp_tgp/pca_postoutlier/', False) 

### 6b. Subcontinental PCA

In [ ]:
# obtain a list of the genetic regions in the dataset - used the unrelated dataset since it had more samples  
regions = mt_unrel['hgdp_tgp_meta']['Genetic']['region'].collect()
regions = list(dict.fromkeys(regions)) # 7 regions - ['EUR', 'AFR', 'AMR', 'EAS', 'CSA', 'OCE', 'MID']

In [ ]:
# set argument values 
subcont_pca_prefix = 'gs://hgdp-1kg/hgdp_tgp/pca_postoutlier/subcont_pca/subcont_pca_' # path for outputs 
overwrite = False 

In [ ]:
# run 'run_pca' function (located above) for each region 
# notebook became slow and got stuck - don't restart it, just let it run and you can follow the progress through the SparkUI
# after checking the desired outputs are generated (GCS bucket) and the run is done (SparkUI), exit the current nb, open a new session, and proceed to the next step
# took roughly 25-27 min  
for i in regions:
    subcont_unrel = mt_unrel.filter_cols(mt_unrel['hgdp_tgp_meta']['Genetic']['region'] == i)  # filter the unrelateds per region
    run_pca(subcont_unrel, i, subcont_pca_prefix, overwrite)

In [ ]:
# run 'project_relateds' function (located above) for each region - took ~3min 
for i in regions:
    loadings = hl.read_table(subcont_pca_prefix + i + '_loadings.ht') # for each region, read in the PCA loadings that were obtained from 'run_pca' function 
    subcont_rel = mt_rel.filter_cols(mt_rel['hgdp_tgp_meta']['Genetic']['region'] == i)  # filter the relateds per region 
    project_individuals(loadings, subcont_rel, i, subcont_pca_prefix, overwrite) 

# 7. Write Out Matrix Table 

In [ ]:
# write out mts of unrelated and related samples separately (post-outlier removal) 

mt_unrel.write('gs://hgdp-1kg/hgdp_tgp/datasets_for_others/lindo/ds_without_outliers/unrelated.mt', overwrite=False) #unrelated mt
mt_rel.write('gs://hgdp-1kg/hgdp_tgp/datasets_for_others/lindo/ds_without_outliers/related.mt', overwrite=False) #related mt